## Check for constructed dataset

In [1]:
import os
import numpy as np
from ipynb.fs.full.Dataset_Construction_Isolated import construct_dataset

lst = os.listdir() 
if ("mushroom_dataset_x.npy" not in lst) or ("mushroom_dataset_y.npy" not in lst):
    print("Dataset files not found, contructing...")
    dataset_x, dataset_y = construct_dataset()
else:
    print("Dataset files found, loading...")
    dataset_x = np.load("mushroom_dataset_x.npy")
    dataset_y = np.load("mushroom_dataset_y.npy")
    print("Done")

Dataset files found, loading...
Done


## Test CNN with Tensorboard

In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime, os
from tensorflow.keras import layers, models, datasets

epochs = 5
steps = 50
batch = 40
    
cnn = models.Sequential([
    tf.keras.Input(shape=(227, 227, 3), batch_size=batch),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

cnn.compile(optimizer="adam", loss="binary_crossentropy",  metrics=["accuracy"])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')  

In [4]:
history = cnn.fit(dataset_x, 
                  dataset_y, validation_split=0.1, epochs=epochs, steps_per_epoch=steps, callbacks=[tensorboard_callback])

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 24s 416ms/step - accuracy: 0.7812 - loss: 0.6923 - val_accuracy: 0.0150 - val_loss: 1.4329
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 396ms/step - accuracy: 0.8674 - loss: 0.3242 - val_accuracy: 0.0550 - val_loss: 1.9355
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 395ms/step - accuracy: 0.9032 - loss: 0.2297 - val_accuracy: 0.0500 - val_loss: 2.0559
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 403ms/step - accuracy: 0.9379 - loss: 0.1600 - val_accuracy: 0.1500 - val_loss: 1.3691
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 395ms/step - accuracy: 0.9383 - loss: 0.1627 - val_accuracy: 0.2000 - val_loss: 2.2798


In [ ]:
%tensorboard --logdir="logs/" --host=127.0.0.1 

## Tensorboard Test

In [ ]:
import tensorflow as tf
import datetime, os

mnist = tf.keras.datasets.mnist

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

def train_model():

  model = create_model()
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')  

  model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

train_model()

## Alternative to Tensorboard logging

In [ ]:
from tensorflow.keras.callbacks import Callback

class StepLogger(Callback):

    def __init__(self, logs={}):
        self.a = []
        
    def on_train_batch_end(self, batch, logs={}):
        self.a.append(logs["accuracy"])